In [1]:
def is_type(obj, obj_type):
    return True if type(obj) is obj_type else False

def exit_with_error(message: str):
    FAILURE = 1
    print(message)
    exit(FAILURE)


class Rules:
    MAX_STEPS = 9
    MAX_THROWS = 9

    ROCK = "r"
    PAPER = "p"
    SCISSOR = "s"
    VALID_SYMBOLS = {ROCK, PAPER, SCISSOR}

    UPPER = "upper"
    LOWER = "lower"
    VALID_TEAMS = {UPPER, LOWER}

    THROW = "THROW"
    SLIDE = "SLIDE"
    SWING = "SWING"
    VALID_ACTIONS = {THROW, SLIDE, SWING}

    HEX_RADIUS = 5
    # range -4 to 4 because map includes (0,0)
    HEX_RANGE = range(-HEX_RADIUS+1, HEX_RADIUS)
 
    
    
class Hex:
    """
    Hexagonal axial coordinates with basic operations and hexagonal
    manhatten distance.
    Thanks to https://www.redblobgames.com/grids/hexagons/ for some
    of the ideas implemented here.
    """

    HEX_RADIUS = Rules.HEX_RADIUS

    r: int
    q: int
        
    @classmethod
    def is_in_boundary(cls, r: int, q: int):
        """
        Determines whether the given coordinate (r,q) or given Hex is within the bounds of our 
        playing board. Inputting a Hex will override any other arguments given.
        """
        if not (is_type(r, int) and is_type(q, int)):
            exit_with_error("Error in Hex.is_in_boundary(): r or q is not int type.")
        if abs(r) >= cls.HEX_RADIUS or abs(q) >= cls.HEX_RADIUS or abs(r+q) >= cls.HEX_RADIUS:
            return False
        return True
    
    
    @classmethod
    def is_valid_coordinate(cls, coordinate: tuple):
        if not is_type(coordinate, tuple): return False
        if len(coordinate) != 2: return False
        return Hex.is_in_boundary(coordinate[0], coordinate[1])
    

    def __init__(self, r: int = None, q: int = None, coordinate: tuple = None):
        if coordinate:
            if not Hex.is_valid_coordinate(coordinate):
                exit_with_error("Error in Hex.__init__(): invalid coordinate.")
            self.r = coordinate[0]
            self.q = coordinate[1]
        else:
            if not Hex.is_in_boundary(r, q):
                exit_with_error("Error in Hex.__init__(): r or q out of boundaries.")
            self.r = r
            self.q = q
        
    def to_tuple(self):
        return (self.r, self.q)

    def __str__(self):
        return str(self.to_tuple())

    @staticmethod
    def dist(hex_1, hex_2):
        """
        Hexagonal manhattan distance between two hex coordinates.
        """
        if not (is_type(hex_1, Hex) and is_type(hex_2, Hex)):
            exit_with_error("Error in Hex.dist(): invalid inputs.")
        delta_r = hex_1.r - hex_2.r
        delta_q = hex_1.q - hex_2.q
        return (abs(delta_r) + abs(delta_q) + abs(delta_r + delta_q)) // 2

    def __add__(self, other_hex):
        # this special method is called when two Hex objects are added with +
        if not is_type(other_hex, Hex):
            exit_with_error("Error in Hex.__add__(): invalid inputs.")
        return Hex(self.r + other_hex.r, self.q + other_hex.q)
    
    def __eq__(self, other_hex):
        if not is_type(other_hex, Hex):
            exit_with_error("Error in Hex.__eq__(): invalid inputs.")
        return self.r == other_hex.r and self.q == other_hex.q
    
    def __ne__(self, other_hex):
        if not is_type(other_hex, Hex):
            exit_with_error("Error in Hex.__ne__(): invalid inputs.")
        return self.r != other_hex.r or self.q != other_hex.q

    def __hash__(self):
        return hash((self.r, self.q))
    
    def adjacents(self):
        """
        Creates a set of neighbouring coordinates to self's coordinate (r,q)
        """
        output = set({})
        r = self.r
        q = self.q
        for item in set({(r+1, q-1),(r+1,q),(r,q+1),(r-1,q+1),(r-1,q),(r,q-1)}):
            if Hex.is_valid_coordinate(item):
                output.add(Hex(coordinate=item))
        return output


class Map:
    HEX_STEPS = [Hex(r, q) for r, q in [(1,-1),(1,0),(0,1),(-1,1),(-1,0),(0,-1)]]
    ALL_HEXES = frozenset(Hex(r, q) for r in Rules.HEX_RANGE for q in Rules.HEX_RANGE if -r-q in Rules.HEX_RANGE)

    _ORD_HEXES = [(r, q) for r in Rules.HEX_RANGE for q in Rules.HEX_RANGE if -r - q in Rules.HEX_RANGE]
    _SET_HEXES = frozenset(_ORD_HEXES)



class Token:
    PAPER = Rules.PAPER
    ROCK = Rules.ROCK
    SCISSOR = Rules.SCISSOR

    hex:    Hex
    symbol: str
    # id: int
    
    BEATS_WHAT = {'r': 's', 'p': 'r', 's': 'p'}
    WHAT_BEATS = {'r': 'p', 'p': 's', 's': 'r'}

    def __init__(self, hex: Hex, symbol: str):
        if not (is_type(hex, Hex) and symbol in Rules.VALID_SYMBOLS):
            exit_with_error("Error in Token.__init__(): invalid inputs.")
        self.hex = hex
        self.symbol = symbol
    
    def to_tuple(self):
        return (self.hex.to_tuple(), self.symbol)
    
    def __str__(self):
        return str(self.to_tuple())

    def move(self, new_hex: Hex):
        if not (is_type(new_hex, Hex)):
            exit_with_error("Error in Token.move(): invalid inputs.")
        self.hex = new_hex

    def is_paper(self):
        return self.symbol == Token.PAPER

    def is_rock(self):
        return self.symbol == Token.ROCK

    def is_scissor(self):
        return self.symbol == Token.SCISSOR

    def beats_what(self):
        return Token.BEATS_WHAT[self.symbol]

    def what_beats(self):
        return Token.WHAT_BEATS[self.symbol]

    def dist(self, other_hex: Hex = None, other_token = None):
        if other_hex != None:
            if not is_type(other_hex, Hex):
                exit_with_error("Error in Token.dist(): invalid other_hex input.")
            return Hex.dist(self.hex, other_hex)
        if other_token != None:
            if not is_type(other_token, Token):
                exit_with_error("Error in Token.dist(): invalid other_token input.")
            return Hex.dist(self.hex, other_token.hex)
        exit_with_error("Error in Token.dist(): no arguments were given.")

    def find_closest_token(self, tokens: list):
        if tokens is None: return None
        for token in tokens:
            if not is_type(token, Token):
                exit_with_error("Error in Token.find_closest_token(): tokens list is invalid.")
        if len(tokens) == 0: return None
        closest = None
        min = -1
        for token in tokens:
            dist = self.dist(other_token = token)
            if min == -1 or dist < min:
                min = dist  
                closest = token     
        return closest



class Action:
    THROW = Rules.THROW
    SLIDE = Rules.SLIDE
    SWING = Rules.SWING

    action_type: str
    token_symbol: str
    from_hex: Hex
    to_hex: Hex
        
    @staticmethod
    def check_valid_action_tuple(action_tuple: tuple):
        # check format of action_tuple
        if not is_type(action_tuple, tuple): return False
        if len(action_tuple) != 3: return False
        # check 1st argument
        if action_tuple[0] not in Rules.VALID_ACTIONS: return False
        #check 2nd argument
        if action_tuple[0] == Action.THROW:
            if action_tuple[1] not in Rules.VALID_SYMBOLS: return False
        else:
            if not is_type(action_tuple[1], tuple): return False
            if not Hex.is_valid_coordinate(action_tuple[1]): return False
        #check 3rd argument
        if not is_type(action_tuple[2], tuple): return False
        if not Hex.is_valid_coordinate(action_tuple[2]): return False
        return True                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

    def __init__(self, action_type: str = None, token_symbol: str = None, from_hex: Hex = None, to_hex: Hex = None, action_tuple: tuple = None):

        # If we were not given an action_tuple, we will create one from the other arguments that are supposed to be given
        # Otherwise continue
        if not action_tuple:
            if not action_type in Rules.VALID_ACTIONS:
                exit_with_error("Error in Action.__init__(): invalid action_type input.")
            # convert from_hex and to_hex into tuples inside of Hexes so that we could use check_valid_action_tuple() function
            if from_hex: 
                if not is_type(from_hex, Hex):
                    exit_with_error("Error in Action.__init__(): invalid from_hex input.")
                from_hex = from_hex.to_tuple()
            if to_hex: 
                if not is_type(to_hex, Hex):
                    exit_with_error("Error in Action.__init__(): invalid to_hex input.")
                to_hex = to_hex.to_tuple()
            # create the action_tuple so we can test its validity
            action_tuple = (action_type, token_symbol, to_hex) if (action_type == Action.THROW) else (action_type, from_hex, to_hex)
        
        # Here we have an action_tuple, we need to check that it's valid
        if not Action.check_valid_action_tuple(action_tuple):
            exit_with_error("Error in Action.__init__(): invalid input arguments.")
        # Here our action_tuple is valid and is ready to be turned into an Action object
        self.action_type = action_tuple[0]
        self.to_hex = Hex(coordinate = action_tuple[2])
        if action_tuple[0] == Action.THROW:
            self.token_symbol = action_tuple[1]
        else:
            self.from_hex = Hex(coordinate = action_tuple[1])

    def is_throw(self):
        return self.action_type == Action.THROW

    def is_slide(self):
        return self.action_type == Action.SLIDE

    def is_swing(self):
        return self.action_type == Action.SWING

    def to_tuple(self):
        if self.is_throw():
            return (self.action_type, self.token_symbol, self.to_hex.to_tuple())
        else:
            return (self.action_type, self.from_hex.to_tuple(), self.to_hex.to_tuple())

    def __str__(self):
        return str(self.to_tuple())


In [2]:

"""
This file was created while referencing the following website:
https://www.annytab.com/a-star-search-algorithm-in-python/
"""

class Node:

    def __init__(self, position: Hex, parent: Hex):
        self.position = position
        self.parent = parent
        self.g = 0  # Estimated distance from start node
        self.h = 0  # Estimated distance to goal node 
        self.f = 0  # g + h
    
    def __eq__(self, other):
        return other.position == self.position

    def __lt__(self, other):
        return other.f > self.f
    
    def __repr__(self):
        return ('({0},{1})'.format(self.position, self.f))

def astar_search(team_dict: dict, team_name: str, start: Token, end: Token, blacklist: list = []):
    """
    Performs A* search
    """
    if team_name not in Rules.VALID_TEAMS:
        exit_with_error("Error in astar_search(): invalid team_name format.")
    for team in Rules.VALID_TEAMS:
        if team not in team_dict:
            exit_with_error("Error in astar_search(): incorrect team_dict dictionary format.")
    
    team = team_dict[team_name]
    open = []
    closed = []

    start_node = Node(start.hex, None)
    goal_node = Node(end.hex, None)

    open.append(start_node)

    while open:
        # Sorting the open list so that the node with the lowest f value is first
        open.sort()
        
        current_node = open.pop(0)
        closed.append(current_node)
        
        # If goal has been reached, backtrack to find the path and return
        if current_node == goal_node:
            path = []
            while current_node != start_node:
                path.append(current_node.position)
                current_node = current_node.parent
            path.append(start.hex)
            return path[::-1]

        x = current_node.position
        
        # Finding actions of current node
        neighbours = [a.to_hex for a in team._move_actions(x, team_dict)]

        for neighbour in neighbours:
            neighbour = Node(neighbour, current_node)

            if neighbour in closed: continue
            if neighbour.position in blacklist: continue

            # Using euclidean distance to calculate heuristics
            neighbour.g = Hex.dist(neighbour.position, start_node.position)
            neighbour.h = Hex.dist(neighbour.position, goal_node.position)
            # neighbour.g = distance.euclidean(neighbour.position, start_node.position)
            # neighbour.h = distance.euclidean(neighbour.position, goal_node.position)
            neighbour.f = neighbour.g + neighbour.h

            if (add_to_open(open,neighbour) == True):
                open.append(neighbour)

    # No paths found
    return None

def find_path_length(team_dict: dict, team_name: str, start: Token, end: Token):
    path = astar_search(team_dict, team_name, start, end)
    return len(path) if path else None

def add_to_open(open, neighbour):
    """
    Checks if a neighbour should be added to open list
    """
    for node in open:
        if neighbour == node and neighbour.f >= node.f:
            # Will not add if there already exists the same node in open that has lower f value
            return False

    return True


def find_attack_moves_for_token(team_dict: dict, team_name: str, start: Token, end: Token):
    moves = []
    blacklist = []
    # team = team_dict[team_name]
    path = astar_search(team_dict, team_name, start, end)
    if path:
        current_path_len = len(path)
    else:
        # print("No path???")
        # print(f'start: {start}')
        # print(f'end: {end}')
        return None

    if len(path) >= 2:
        while path:
            if len(path) == current_path_len:
                moves.append(path[1])
                blacklist.append(path[1])
                path = astar_search(team_dict, team_name, start, end, blacklist=blacklist)
            else: break
    return moves



In [3]:
import random
import copy
import numpy as np


UPPER = Rules.UPPER
LOWER = Rules.LOWER
OPPOSITE_HEXES = {(0,-1): Hex(0,1), (1,-1):Hex(-1,1), (1,0):Hex(-1,0), (0,1):Hex(0,-1), (-1,1):Hex(1,-1), (-1,0):Hex(1,0)}
ATTACK = "attack"
RUN = "run"
OVERLAP = "overlap"

class Team:
    team_name: str
    throws_remaining: int
    active_tokens: list

    """----> Hold the last 5 moves that we made"""
    previous_moves: list

    """----> Keep track of how many states have passed since we last moved each token."""

    def __init__(self, team_name: str, active_tokens: list = None, throws_remaining: int = None):
        if team_name not in Rules.VALID_TEAMS:
            exit_with_error("Error in Team.__init__(): invalid team_name input.")
        self.team_name = team_name
        if throws_remaining:
            self.throws_remaining = copy.deepcopy(throws_remaining)
        else:
            self.throws_remaining = Rules.MAX_THROWS
        if active_tokens:
            self.active_tokens = copy.deepcopy(active_tokens)
        else:
            self.active_tokens = []
        self.previous_moves = []

    def __str__(self):
        print("Team " + self.team_name + ", " + "throws_remaining: " + str(self.throws_remaining) + "\nActive tokens: ", end = '')
        [print(str(token) + ", ", end='') for token in self.active_tokens]
        return ''
     
    def get_token_at(self, hex: Hex):
        """ Returns the first token found at specified hex for this team. """
        if not is_type(hex, Hex):
            exit_with_error("Error in Team.get_token_at(): hex input is not a Hex.")
        for token in self.active_tokens:
            if token.hex == hex:
                return token
        return None

    def get_tokens_at(self, hex: Hex):
        """ Returns all the tokens found at specified hex for this team. """
        if not is_type(hex, Hex):
            exit_with_error("Error in Team.get_tokens_at(): hex input is not a Hex.")
        return [token for token in self.active_tokens if token.hex == hex]

    def exists_token_at(self, hex: Hex):
        """Check if there exists a token at specified hex for this team."""
        if not is_type(hex, Hex):
            exit_with_error("Error in Team.exists_token_at(): hex input is not a Hex.")
        if self.get_token_at(hex):
            return True
        return False

    def get_rock_tokens(self):
        return [token for token in self.active_tokens if token.symbol == Token.ROCK]

    def get_paper_tokens(self):
        return [token for token in self.active_tokens if token.symbol == Token.PAPER]

    def get_scissor_tokens(self):
        return [token for token in self.active_tokens if token.symbol == Token.SCISSOR]

    def get_num_rock(self):
        return len(self.get_rock_tokens())

    def get_num_paper(self):
        return len(self.get_paper_tokens())

    def get_num_scissor(self):
        return len(self.get_scissor_tokens())

    def get_tokens_of_type(self, token_type: str):
        if token_type not in Rules.VALID_SYMBOLS:
            exit_with_error("Error in Team.get_tokens_of_type(): token_type invalid.")
        if token_type == Token.ROCK:
            return self.get_rock_tokens()
        if token_type == Token.PAPER:
            return self.get_paper_tokens()
        if token_type == Token.SCISSOR:
            return self.get_scissor_tokens()

    def has_active_token(self, token_type: str):
        if token_type not in Rules.VALID_SYMBOLS:
            exit_with_error("Error in Team.has_active_token(): token_type invalid.")
        if self.get_tokens_of_type(token_type):
            return True
        return False

    def get_num_dups(self, token_type: str):
        if token_type not in Rules.VALID_SYMBOLS:
            exit_with_error("Error in Team.get_num_dups(): token_type invalid.")
        return len(self.get_tokens_of_type(token_type))

    def decrease_throw(self, by=1):
        self.throws_remaining -= by

    def first_move(self):
        s = [Token.ROCK, Token.PAPER, Token.SCISSOR]
        i = random.randint(0,2)
        reach = max(Rules.HEX_RANGE)
        throw_hex = Hex(reach,-(reach)/2)
        if self.team_name == UPPER:
            return Action(action_type = Action.THROW, token_symbol = s[i], to_hex = throw_hex)
        if self.team_name == LOWER:
            throw_hex.invert()
            return Action(action_type = Action.THROW, token_symbol = s[i], to_hex = throw_hex)

    def generate_occupied_hexes(self):
        xs = [x.hex for x in self.active_tokens]
        occupied_hexes = set(xs)
        return occupied_hexes
    
    def generate_dangerous_hexes(self, team_dict: dict):
        for team_name in Rules.VALID_TEAMS:
            if team_name not in team_dict:
                exit_with_error("Error in Team.generate_dangerous_hexes(): incorrect team_dict dictionary format.")
            if not is_type(team_dict[team_name], Team):
                exit_with_error("Error in Team.generate_dangerous_hexes(): incorrect team_dict dictionary format.")

        enemy = LOWER if self.team_name == UPPER else UPPER
        enemy_rocks = set([x.hex for x in team_dict[enemy].get_rock_tokens()])
        enemy_papers = set([x.hex for x in team_dict[enemy].get_paper_tokens()])
        enemy_scissors = set([x.hex for x in team_dict[enemy].get_scissor_tokens()])
        dangerous_hexes = {Token.ROCK: enemy_papers, Token.PAPER: enemy_scissors, Token.SCISSOR: enemy_rocks}
        return dangerous_hexes

    def generate_enemy_defeatable(self, team_dict: dict):
        enemy_team = team_dict[UPPER] if self.team_name == LOWER else team_dict[LOWER]
        enemy_rocks = enemy_team.get_tokens_of_type(Rules.ROCK)
        enemy_papers = enemy_team.get_tokens_of_type(Rules.PAPER)
        enemy_scissors = enemy_team.get_tokens_of_type(Rules.SCISSOR)
        defeatable_tokens = {Rules.PAPER: enemy_rocks, Rules.ROCK: enemy_scissors, Rules.SCISSOR: enemy_papers}
        return defeatable_tokens

    def generate_enemy_defeated_by(self, team_dict: dict):
        enemy_team = team_dict[UPPER] if self.team_name == LOWER else team_dict[LOWER]
        enemy_rocks = enemy_team.get_tokens_of_type(Rules.ROCK)
        enemy_papers = enemy_team.get_tokens_of_type(Rules.PAPER)
        enemy_scissors = enemy_team.get_tokens_of_type(Rules.SCISSOR)
        defeated_by_tokens = {Rules.PAPER: enemy_scissors, Rules.ROCK: enemy_papers, Rules.SCISSOR: enemy_rocks}
        return defeated_by_tokens





























    def _move_actions(self, x: Hex, team_dict: dict):
        for team_name in Rules.VALID_TEAMS:
            if team_name not in team_dict:
                exit_with_error("Error in Team._move_actions(): incorrect team_dict dictionary format.")
            if not is_type(team_dict[team_name], Team):
                exit_with_error("Error in Team._move_actions(): incorrect team_dict dictionary format.")
        if not is_type(x, Hex):
            exit_with_error("Error in Team._move_actions(): x input is not a Hex.")


        occupied_hexes = self.generate_occupied_hexes()
        dangerous_hexes = self.generate_dangerous_hexes(team_dict)
        token = self.get_token_at(x)
        # find all adjacent hexes that are not occupied by ally tokens
        adjacents_x = set(x.adjacents()) - occupied_hexes 
        if token:
            adjacents_x -= dangerous_hexes[token.symbol]
        # find all hexes that are occupied 
        # x_token = self.get_token_at(x)
        actions = []
        for y in adjacents_x:
            actions.append(Action(action_type = "SLIDE", from_hex = x, to_hex = y))
            if y in occupied_hexes:
                opposites_y = y.adjacents() - adjacents_x - occupied_hexes - {x}
                for z in opposites_y:
                    actions.append(Action(action_type="SWING", from_hex = x, to_hex =z))
        # print(f'slides/swings: {len(actions)}')
        # random.shuffle(actions)
        return actions

        
    
    def generate_throw_zone(self, team_dict: dict, token_type: str):
        for team_name in Rules.VALID_TEAMS:
            if team_name not in team_dict:
                exit_with_error("Error in Team.generate_throw_zone(): incorrect team_dict dictionary format.")
            if not is_type(team_dict[team_name], Team):
                exit_with_error("Error in Team.generate_throw_zone(): incorrect team_dict dictionary format.")
        if token_type not in Rules.VALID_SYMBOLS:
            exit_with_error("Error in Team.generate_throw_zone(): invalid token_type.")

        throws = Rules.MAX_THROWS - self.throws_remaining
        sign = -1 if self.team_name == LOWER else 1
        dangerous_hexes = self.generate_dangerous_hexes(team_dict)
        radius_range = Rules.HEX_RADIUS - 1
        throw_zone = {Hex(r, q) for r, q in Map._SET_HEXES if (sign * r >= radius_range - throws and not self.exists_token_at(Hex(r, q)))}
        throw_zone -= dangerous_hexes[token_type]
        
        return throw_zone

    
    '''
    Only generate like 3 throw actions one for each type
    '''
    # def _throw_actions(self, team_dict: dict):
    #     for team_name in Rules.VALID_TEAMS:
    #         if team_name not in team_dict:
    #             exit_with_error("Error in Team._throw_actions(): incorrect team_dict dictionary format.")
    #         if not is_type(team_dict[team_name], Team):
    #             exit_with_error("Error in Team._throw_actions(): incorrect team_dict dictionary format.")

    #     throw_actions = []
    #     if self.throws_remaining > 0:
    #         # Only throw this type if we do NOT already have it on the board
    #         throw_types = {symbol for symbol in Rules.VALID_SYMBOLS if not self.has_active_token(symbol)}
    #         for _s in throw_types:
    #             throw_zone = self.generate_throw_zone(team_dict, _s)
    #             for x in throw_zone:
    #                 throw_actions.append(Action(action_type = Action.THROW, token_symbol=_s, to_hex = x))
    #     # print(f'throws: {len(throw_actions)}')
    #     # random.shuffle(throw_actions)
    #     return throw_actions
    def _throw_actions(self, team_dict: dict, token_symbol: str):
        enemy_team = team_dict[UPPER] if self.team_name == LOWER else team_dict[LOWER]
        best_throw = None
        throw_action = None
        min_dist = -1

        if self.throws_remaining == 0:
            return None
        
        if self.throws_remaining > 0:
            throw_zone = self.generate_throw_zone(team_dict, token_symbol)
            for throw in throw_zone:
                killable_enemies = enemy_team.get_tokens_of_type(Token.BEATS_WHAT[token_symbol])
                for enemy in killable_enemies:  
                    dist = Hex.dist(throw, enemy.hex)
                    if min_dist == -1 or dist < min_dist:
                        min_dist = dist
                        best_throw = throw
            if best_throw:
                throw_action = Action(action_type = Rules.THROW, token_symbol = token_symbol, to_hex=best_throw)
        return throw_action


    def token_run_away_from(self, ally_token, opp_token, team_dict: dict):
        occupied_hexes = self.generate_occupied_hexes()
        dangerous_hexes = self.generate_dangerous_hexes(team_dict)
        x = ally_token.hex
        # find all adjacent hexes that are not occupied by ally tokens
        adjacents_x = set(x.adjacents()) - occupied_hexes 
        adjacents_x -= dangerous_hexes[ally_token.symbol]
        # find all hexes that are occupied 
        # x_token = self.get_token_at(x)
        moves = set({})
        for y in adjacents_x:
            moves.add(y)
            if y in occupied_hexes:
                opposites_y = y.adjacents() - adjacents_x - occupied_hexes - {x}
                for z in opposites_y:
                    moves.add(z)
        
        hex_avoid = opp_token.hex
        good_moves = []
        max_dist = -1
        for hex in moves:
            if max_dist == -1 or Hex.dist(hex,hex_avoid) > max_dist:
                good_moves = [hex]
                max_dist = Hex.dist(hex,hex_avoid)
            elif Hex.dist(hex,hex_avoid) == max_dist:
                good_moves.append(hex)
        return good_moves

    def generate_run_actions(self, team_dict: dict):
        defeated_by_tokens = self.generate_enemy_defeated_by(team_dict)
        actions = []
        for ally in self.active_tokens:
            scary_dudes = defeated_by_tokens[ally.symbol]
            closest_enemy = ally.find_closest_token(scary_dudes)
            if closest_enemy: 
                run_moves = self.token_run_away_from(ally, closest_enemy, team_dict)
                if not run_moves: continue
                new_dist = Hex.dist(run_moves[0], ally.hex)
                if new_dist == 1 or new_dist > Hex.dist(ally.hex, closest_enemy.hex):
                    for move in run_moves:
                        new_action = Action.create_action_from_path(ally.hex, move)
                        actions.append(new_action) 

        # print("Runnnnn")
        # for action in actions: print(action, end=', ')
        # print('\n')
        return actions                

    def generate_attack_actions(self, team_dict: dict):
        defeatable_tokens = self.generate_enemy_defeatable(team_dict)
        actions = []
        for ally in self.active_tokens:
            # Attack actions
            if defeatable_tokens[ally.symbol]:
                for enemy in defeatable_tokens[ally.symbol]:
                    moves = find_attack_moves_for_token(team_dict, self.team_name, ally, enemy)
                    if moves:
                        for move in moves:
                            new_action = Action.create_action_from_path(ally.hex, move)
                            actions.append(new_action) 
        # print("Actacckkk:")
        # for action in actions: print(action, end=', ')
        # print('\n')
        '''MAYBE: implement a counter for attack options and if there are overlaps, that attack gets a higher weighting'''
        return actions

    def generate_throw_actions(self, team_dict: dict):
        actions = []
        # Best throw action for each token symbol
        for _s in Rules.VALID_SYMBOLS:
            throw_action = self._throw_actions(team_dict, _s)
            if throw_action:
                # print(f'thr: {throw_action}')
                actions.append(throw_action)
        # print("Throwwww")
        # for action in actions: print(action, end=', ')
        # print('\n')
        return actions

    def find_next_move_for_token(self, ally: Token, team_dict: dict):
        defeatable_tokens = self.generate_enemy_defeatable(team_dict)
        defeated_by_tokens = self.generate_enemy_defeated_by(team_dict)
        run_actions = []
        attack_actions = []
        scary_dudes = defeated_by_tokens[ally.symbol]
        closest_enemy = ally.find_closest_token(scary_dudes)
        # Find run actions
        if closest_enemy: 
            run_moves = self.token_run_away_from(ally, closest_enemy, team_dict)
            if run_moves:
                new_dist = Hex.dist(run_moves[0], ally.hex)
                if new_dist == 1 or new_dist > Hex.dist(ally.hex, closest_enemy.hex):
                    for move in run_moves:
                        new_action = Action.create_action_from_path(ally.hex, move)
                        run_actions.append(new_action)
        # Find attack actions
        if defeatable_tokens[ally.symbol]:
            for enemy in defeatable_tokens[ally.symbol]:
                attack_moves = find_attack_moves_for_token(team_dict, self.team_name, ally, enemy)
                if attack_moves:
                    for move in attack_moves:
                        new_action = Action.create_action_from_path(ally.hex, move)
                        attack_actions.append(new_action)
        overlap = Action.find_overlap_actions(run_actions, attack_actions)
        return {ATTACK: attack_actions, RUN: run_actions, OVERLAP: overlap}
        

    def generate_move_actions(self, team_dict: dict):
        attack_actions = []
        run_actions = []
        for ally in self.active_tokens:
            token_actions = self.find_next_move_for_token(ally, team_dict)
            if token_actions[OVERLAP]:
                optimal_move_found = False
                for action in token_actions[OVERLAP]:
                    if Hex.dist(action.from_hex, action.to_hex) == 1:
                        optimal_move_found = True
                        attack_actions.append(action)
                if optimal_move_found: continue
            attack_actions += token_actions[ATTACK]
            run_actions += token_actions[RUN]
        return {ATTACK: attack_actions, RUN: run_actions}









            
    
    
    def generate_actions(self, team_dict: dict):
        """
        Generate all available actions.
        """
        for team_name in Rules.VALID_TEAMS:
            if team_name not in team_dict:
                exit_with_error("Error in Team.generate_actions(): incorrect team_dict dictionary format.")
            if not is_type(team_dict[team_name], Team):
                exit_with_error("Error in Team.generate_actions(): incorrect team_dict dictionary format.")

        occupied_hexes = self.generate_occupied_hexes()
        actions = []
        for x in occupied_hexes:
            actions += self._move_actions(x, team_dict)
        
        # Best throw action for each token symbol
        for _s in Rules.VALID_SYMBOLS:
            throw_action = self._throw_actions(team_dict, _s)
            if throw_action:
                actions.append(throw_action)
        
        # actions += self._throw_actions(team_dict)
        # random.shuffle(actions)
        return actions
    
    # def create_action_from_path(self, token, end:Hex):
    #     move_type = Rules.SLIDE if Hex.dist(token.hex, end) == 1 else Rules.SWING

    #     new_action = Action(action_type = move_type, from_hex=token.hex, to_hex=end)
    #     return new_action

    def generate_good_actions(self, team_dict: dict):
        """
        Good actions include:
        - Chasing a killable opponent
        - Throwing on/close to a killable opponent
        - Running from a dangerous opponent
        """
        
        for team_name in Rules.VALID_TEAMS:
            if team_name not in team_dict:
                exit_with_error("Error in Team.generate_actions(): incorrect team_dict dictionary format.")
            if not is_type(team_dict[team_name], Team):
                exit_with_error("Error in Team.generate_actions(): incorrect team_dict dictionary format.")

        actions = []

        # Move actions
        attack_actions = self.generate_attack_actions(team_dict)
        run_actions = self.generate_run_actions(team_dict)
        
        # If there is overlap, only consider overlap if dist < 1
        overlap = Action.find_overlap_actions(attack_actions, run_actions)
        if overlap:
            actions += overlap
        else:
            actions += attack_actions
            actions += run_actions

        # Throw actions
        actions += self.generate_throw_actions(team_dict)
        
        # If no good actions, random move
        if len(actions) == 0:
            random_actions = self.generate_actions(team_dict)
            i = np.random.randint(0, len(random_actions)-1)
            actions.append(random_actions[int(i)])

        return actions


    def determine_closest_kill(self, team_dict: dict):
        for team_name in Rules.VALID_TEAMS:
            if team_name not in team_dict:
                exit_with_error("Error in Team.determine_closest_kill(): incorrect team_dict dictionary format.")
            if not is_type(team_dict[team_name], Team):
                exit_with_error("Error in Team.determine_closest_kill(): incorrect team_dict dictionary format.")

        enemy_team = team_dict[UPPER] if self.team_name == LOWER else team_dict[LOWER]
        closest_pair = None
        min_dist = -1
        
        for token in self.active_tokens:
            enemy_tokens = enemy_team.get_tokens_of_type(token.beats_what())
            
            enemy_token = token.find_closest_token(enemy_tokens)
            if enemy_token:
                dist = token.dist(other_token = enemy_token)
                if min_dist == -1 or dist < min_dist:
                    min_dist = dist
                    closest_pair = (token, enemy_token)
    
        return closest_pair, min_dist

    def determine_closest_threat(self, team_dict: dict):

        enemy_team = team_dict[UPPER] if self.team_name == LOWER else team_dict[LOWER]
        closest_pair = None
        min_dist = -1
        
        for token in enemy_team.active_tokens:
            ally_tokens = self.get_tokens_of_type(token.beats_what())
            
            ally_token = token.find_closest_token(ally_tokens)
            if ally_token:
                dist = token.dist(other_token = ally_token)
                if min_dist == -1 or dist < min_dist:
                    min_dist = dist
                    closest_pair = (token, ally_token)
        return closest_pair, min_dist

    def determine_best_throw(self, team_dict: dict):
        for team_name in Rules.VALID_TEAMS:
            if team_name not in team_dict:
                exit_with_error("Error in Team.determine_best_throw(): incorrect team_dict dictionary format.")
            if not is_type(team_dict[team_name], Team):
                exit_with_error("Error in Team.determine_best_throw(): incorrect team_dict dictionary format.")

        # Distance of throw zone hexes to enemy tokens
        # Choose minimal distance
        enemy_team = team_dict[UPPER] if self.team_name == LOWER else team_dict[LOWER]
        best_throw = None
        best_throw_symbol = None
        throw_action = None
        min_dist = -1

        if self.throws_remaining == 0:
            return None

        throw_types = {symbol for symbol in Rules.VALID_SYMBOLS if not self.has_active_token(symbol)}
        
        for _s in throw_types:
            throw_zone = self.generate_throw_zone(team_dict, _s)
            for throw in throw_zone:
                killable_enemies = enemy_team.get_tokens_of_type(Token.BEATS_WHAT[_s])
                for enemy in killable_enemies:  
                    dist = Hex.dist(throw, enemy.hex)
                    if min_dist == -1 or dist < min_dist:
                        min_dist = dist
                        best_throw = throw
                        best_throw_symbol = _s
        if best_throw:
            throw_action = Action(action_type = Rules.THROW, token_symbol = best_throw_symbol, to_hex=best_throw)
        
        return throw_action, min_dist



    # def determine_closest_guardian_ally():
    #     # Once you find this, 
    #     return closest_guardian_ally



In [4]:
import itertools
import math
import copy


UPPER = Rules.UPPER
LOWER = Rules.LOWER

class Board:
    team_upper: Team
    team_lower: Team
    team_dict: dict
    upper_tokens: list
    lower_tokens: list

    def __init__(self, team_upper: Team = None, team_lower: Team = None):
        if team_upper is not None and team_lower is not None:
            self.team_upper = Team(team_name = UPPER, active_tokens=team_upper.active_tokens, throws_remaining=team_upper.throws_remaining)
            self.team_lower = Team(team_name = LOWER, active_tokens=team_lower.active_tokens, throws_remaining=team_lower.throws_remaining)
            # self.team_upper = copy.deepcopy(team_upper)
            # self.team_lower = copy.deepcopy(team_lower)
        else:
            if team_upper is not None or team_lower is not None:
                exit_with_error("Error in Board.__init__(): one of the arguments (team_upper/team_lower) is None.")
            self.team_upper = Team(UPPER)
            self.team_lower = Team(LOWER)

        self.team_dict = {UPPER: self.team_upper, LOWER: self.team_lower}
        self.upper_tokens = self.team_upper.active_tokens
        self.lower_tokens = self.team_lower.active_tokens

     
    def get_team(self, team_name: str):
        if team_name not in self.team_dict:
            exit_with_error("Error in Board.get_team(): team name undefined.")
        return self.team_dict[team_name]
    

    def successor(self, actions: dict):
        for team_name in Rules.VALID_TEAMS:
            if team_name not in actions:
                exit_with_error("Error in Team.generate_dangerous_hexes(): incorrect team_dict format.")
            if not is_type(actions[team_name], Action):
                exit_with_error("Error in Team.generate_dangerous_hexes(): incorrect team_dict format.")        

        for team_name in actions:
            team = self.team_dict[team_name]
            action = actions[team_name]
            if action.is_throw():
                new_token = Token(action.to_hex, action.token_symbol)
                team.active_tokens.append(new_token)
                team.decrease_throw()
            else:
                to_move = team.get_token_at(action.from_hex)
                # print(action.from_hex)
                # print(to_move)
                to_move.move(action.to_hex)  

        # where tokens clash, do battle
        # TODO: only necessary to check this at destinations of actions
        # (but then will have to find another way to fill the lists)
        safe_upper_tokens = []
        safe_lower_tokens = []
        for x in Map.ALL_HEXES:
            ups_at_x = [t for t in self.upper_tokens if t.hex == x]
            lws_at_x = [t for t in self.lower_tokens if t.hex == x]
            symbols = {t.symbol for t in ups_at_x + lws_at_x}
            if len(symbols) > 1:
                for _s in symbols:
                    k = Token.BEATS_WHAT[_s]
                    ups_at_x = [t for t in ups_at_x if t.symbol != k]
                    lws_at_x = [t for t in lws_at_x if t.symbol != k]
            safe_upper_tokens.extend(ups_at_x)
            safe_lower_tokens.extend(lws_at_x)

        self.team_upper.active_tokens = safe_upper_tokens
        self.team_lower.active_tokens = safe_lower_tokens
        self.upper_tokens = self.team_upper.active_tokens
        self.lower_tokens = self.team_lower.active_tokens

    def get_tokens(self, team_name: str, token_type: str):
        if team_name not in self.team_dict or token_type not in Rules.VALID_SYMBOLS:
            exit_with_error("Error in Board.get_tokens(): invalid inputs.")
        return [token for token in self.team_dict[team_name].active_tokens if token.symbol == token_type]

    def evaluate_token(self, token: Token, team_name: str):
        score = 0
        WEIGHT = 1/4

        opp_team = self.team_upper if (team_name == LOWER) else self.team_lower

        closest_defeatable = token.find_closest_token(opp_team.get_tokens_of_type(token.beats_what()))
        closest_defeated_by = token.find_closest_token(opp_team.get_tokens_of_type(token.what_beats()))
         
        if closest_defeatable:
            assert(Hex.dist(token.hex, closest_defeatable.hex)+1 > 0)
            score -= math.log(Hex.dist(token.hex, closest_defeatable.hex)+1)
        
        if closest_defeated_by:
            # print(token.symbol)
            # print(token.hex)
            # print(closest_defeated_by.hex)
            assert(Hex.dist(token.hex, closest_defeated_by.hex)+1 > 0)
            score += WEIGHT * math.log(Hex.dist(token.hex, closest_defeated_by.hex)+1)

        return score


    # def evaluate(self, team: Team):
    #     score = 0
    #     opp_team = self.team_upper if (team.team_name == LOWER) else self.team_lower
        
    #     closest_kill, kill_dist = team.determine_closest_kill(self.team_dict)
    #     closest_threat, threat_dist = team.determine_closest_threat(self.team_dict)

    #     if kill_dist > 0:
    #         score -= kill_dist
    #     if threat_dist > 0:
    #         score += (1/2) * threat_dist

    #     score += team.throws_remaining * 1
    #     score += (Rules.MAX_THROWS - opp_team.throws_remaining) - len(opp_team.active_tokens)
    #     score -= ((Rules.MAX_THROWS - team.throws_remaining) - len(team.active_tokens))
        
    #     return score


    def evaluate(self, team: Team):
        score = 0
        enemy_team = self.team_upper if (team.team_name == LOWER) else self.team_lower

        enemy_defeatable = team.generate_enemy_defeatable(self.team_dict)
        enemy_defeated_by = team.generate_enemy_defeated_by(self.team_dict)

        score += team.throws_remaining * 1
        score += len(team.active_tokens)
        if team.get_num_dups(Rules.ROCK):
            score = score - team.get_num_dups(Rules.ROCK) + 1
        if team.get_num_dups(Rules.PAPER):
            score = score - team.get_num_dups(Rules.PAPER) + 1
        if team.get_num_dups(Rules.SCISSOR):
            score = score - team.get_num_dups(Rules.SCISSOR) + 1

        weight_dist = {0: 1.22, 1: 1, 2: 0.80, 3: 0.62, 4: 0.46, 5: 0.32, 6: 0.20, 7: 0.10, 8: 0.02, 9: 0}
        for token in team.active_tokens:
            if enemy_defeatable[token.symbol]:
                for enemy_token in enemy_defeatable[token.symbol]:
                    path_length = find_path_length(self.team_dict, team.team_name, token, enemy_token)
                    if path_length:
                        dist_kill = path_length - 1
                        if dist_kill < 10: weight = weight_dist[dist_kill]
                        else: weight = 0
                        score += ((9 - dist_kill) * weight)
            if enemy_defeated_by[token.symbol]:
                for enemy_token in enemy_defeated_by[token.symbol]:
                    path_length = find_path_length(self.team_dict, team.team_name, token, enemy_token)
                    if path_length:
                        dist_threat = path_length - 1
                        if dist_threat < 10: weight = weight_dist[dist_threat]
                        else: weight = 0
                        score += (dist_threat * weight)

        enemy_active = len(enemy_team.active_tokens)
        enemy_num_defeatable = sum([len(enemy_defeatable[_s]) for _s in Rules.VALID_SYMBOLS])
        enemy_num_invincible = enemy_active - enemy_num_defeatable
        
        score += (9 - enemy_num_invincible) * 1.5

        """
        score += throws_remaining * 1.1
        score += tokens_active
        if find_num_dups(R)
            score -= find_num_dups(R) + 1
        if find_num_dups(P)
            score -= find_num_dups(P) + 1
        if find_num_dups(S)
            score -= find_num_dups(S) + 1

        weight_dist = {1: 1, 2: 0.80, 3: 0.62, 4: 0.46, 5: 0.32, 6: 0.20, 7: 0.10, 8: 0.02, 9: 0}
        if token_kills: (make find_token_kills)
            score += (9 - dist_kill) * weight_dist[dist_kill]
        if token_threats: (make find_token_threats)
            score += (dist_threat) * weight_dist[dist_threat]
            

        RUN:
        only run when opponent is within 3 steps
        if there is an overlap in run and attack, take that
        if there is no attack, find opponent token of the same type
        if standing on opponent token of same type: safe, do not move unless there is a target to kill


        CUT IN GREEDYS PATH: 
        if already in path, move towards kill, else cut the way.
        if next to target and haven't killed this target in 3 turns, try different method.
        score 
        """

        return score

        

    def print_board(self, board_dict, message="", compact=True, ansi=False, **kwargs):
        """
        For help with visualisation and debugging: output a board diagram with
        any information you like (tokens, heuristic values, distances, etc.).
        Arguments:
        board_dict -- A dictionary with (r, q) tuples as keys (following axial
            coordinate system from specification) and printable objects (e.g.
            strings, numbers) as values.
            This function will arrange these printable values on a hex grid
            and output the result.
            Note: At most the first 5 characters will be printed from the string
            representation of each value.
        message -- A printable object (e.g. string, number) that will be placed
            above the board in the visualisation. Default is "" (no message).
        ansi -- True if you want to use ANSI control codes to enrich the output.
            Compatible with terminals supporting ANSI control codes. Default
            False.
        compact -- True if you want to use a compact board visualisation,
            False to use a bigger one including axial coordinates along with
            the printable information in each hex. Default True (small board).
        
        Any other keyword arguments are passed through to the print function.
        Example:
            >>> board_dict = {
            ...     ( 0, 0): "hello",
            ...     ( 0, 2): "world",
            ...     ( 3,-2): "(p)",
            ...     ( 2,-1): "(S)",
            ...     (-4, 0): "(R)",
            ... }
            >>> print_board(board_dict, "message goes here", ansi=False)
            # message goes here
            #              .-'-._.-'-._.-'-._.-'-._.-'-.
            #             |     |     |     |     |     |
            #           .-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
            #          |     |     | (p) |     |     |     |
            #        .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
            #       |     |     |     | (S) |     |     |     |
            #     .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
            #    |     |     |     |     |     |     |     |     |
            #  .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
            # |     |     |     |     |hello|     |world|     |     |
            # '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
            #    |     |     |     |     |     |     |     |     |
            #    '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
            #       |     |     |     |     |     |     |     |
            #       '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
            #          |     |     |     |     |     |     |
            #          '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
            #             | (R) |     |     |     |     |
            #             '-._.-'-._.-'-._.-'-._.-'-._.-'
        """
        if compact:
            template = """# {00:}
    #              .-'-._.-'-._.-'-._.-'-._.-'-.
    #             |{57:}|{58:}|{59:}|{60:}|{61:}|
    #           .-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
    #          |{51:}|{52:}|{53:}|{54:}|{55:}|{56:}|
    #        .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
    #       |{44:}|{45:}|{46:}|{47:}|{48:}|{49:}|{50:}|
    #     .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
    #    |{36:}|{37:}|{38:}|{39:}|{40:}|{41:}|{42:}|{43:}|
    #  .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
    # |{27:}|{28:}|{29:}|{30:}|{31:}|{32:}|{33:}|{34:}|{35:}|
    # '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
    #    |{19:}|{20:}|{21:}|{22:}|{23:}|{24:}|{25:}|{26:}|
    #    '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
    #       |{12:}|{13:}|{14:}|{15:}|{16:}|{17:}|{18:}|
    #       '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
    #          |{06:}|{07:}|{08:}|{09:}|{10:}|{11:}|
    #          '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
    #             |{01:}|{02:}|{03:}|{04:}|{05:}|
    #             '-._.-'-._.-'-._.-'-._.-'-._.-'"""
        else:
            template = """# {00:}
    #                  ,-' `-._,-' `-._,-' `-._,-' `-._,-' `-.
    #                 | {57:} | {58:} | {59:} | {60:} | {61:} |
    #                 |  4,-4 |  4,-3 |  4,-2 |  4,-1 |  4, 0 |
    #              ,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-.
    #             | {51:} | {52:} | {53:} | {54:} | {55:} | {56:} |
    #             |  3,-4 |  3,-3 |  3,-2 |  3,-1 |  3, 0 |  3, 1 |
    #          ,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-.
    #         | {44:} | {45:} | {46:} | {47:} | {48:} | {49:} | {50:} |
    #         |  2,-4 |  2,-3 |  2,-2 |  2,-1 |  2, 0 |  2, 1 |  2, 2 |
    #      ,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-.
    #     | {36:} | {37:} | {38:} | {39:} | {40:} | {41:} | {42:} | {43:} |
    #     |  1,-4 |  1,-3 |  1,-2 |  1,-1 |  1, 0 |  1, 1 |  1, 2 |  1, 3 |
    #  ,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-.
    # | {27:} | {28:} | {29:} | {30:} | {31:} | {32:} | {33:} | {34:} | {35:} |
    # |  0,-4 |  0,-3 |  0,-2 |  0,-1 |  0, 0 |  0, 1 |  0, 2 |  0, 3 |  0, 4 |
    #  `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-'
    #     | {19:} | {20:} | {21:} | {22:} | {23:} | {24:} | {25:} | {26:} |
    #     | -1,-3 | -1,-2 | -1,-1 | -1, 0 | -1, 1 | -1, 2 | -1, 3 | -1, 4 |
    #      `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-'
    #         | {12:} | {13:} | {14:} | {15:} | {16:} | {17:} | {18:} |
    #         | -2,-2 | -2,-1 | -2, 0 | -2, 1 | -2, 2 | -2, 3 | -2, 4 |
    #          `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-'
    #             | {06:} | {07:} | {08:} | {09:} | {10:} | {11:} |
    #             | -3,-1 | -3, 0 | -3, 1 | -3, 2 | -3, 3 | -3, 4 |   key:
    #              `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-'     ,-' `-.
    #                 | {01:} | {02:} | {03:} | {04:} | {05:} |       | input |
    #                 | -4, 0 | -4, 1 | -4, 2 | -4, 3 | -4, 4 |       |  r, q |
    #                  `-._,-' `-._,-' `-._,-' `-._,-' `-._,-'         `-._,-'"""
        # prepare the provided board contents as strings, formatted to size.
        reach = Rules.HEX_RANGE
        cells = []
        for rq in [(r,q) for r in reach for q in reach if -r-q in reach]:
            if rq in board_dict and board_dict[rq]:
                cell = ""
                for token in board_dict[rq]:
                    cell = cell + str(token) + ','
                cell = cell[:-1].center(5)
                if ansi:
                    # put contents in bold
                    cell = f"\033[1m{cell}\033[0m"
            else:
                cell = "     " # 5 spaces will fill a cell
            cells.append(cell)
        # prepare the message, formatted across multiple lines
        multiline_message = "\n# ".join(message.splitlines())
        # fill in the template to create the board drawing, then print!
        board = template.format(multiline_message, *cells)
        print(board, **kwargs)



    def __str__(self):
        print(self.team_upper)
        print(self.team_lower)

        board_dict = {}
        for team_name in self.team_dict:
            team = self.team_dict[team_name]
            if not team.active_tokens: continue
            for token in team.active_tokens:
                _s = token.symbol
                if team_name == UPPER: _s = _s.upper()
                position = token.hex.to_tuple()
                if position in board_dict:
                    board_dict[position].append(_s)
                else:
                    board_dict[position] = [_s]

        self.print_board(board_dict, compact=True)
        return ''

In [5]:
def print_board(board_dict, message="", compact=True, ansi=False, **kwargs):
    """
    For help with visualisation and debugging: output a board diagram with
    any information you like (tokens, heuristic values, distances, etc.).
    Arguments:
    board_dict -- A dictionary with (r, q) tuples as keys (following axial
        coordinate system from specification) and printable objects (e.g.
        strings, numbers) as values.
        This function will arrange these printable values on a hex grid
        and output the result.
        Note: At most the first 5 characters will be printed from the string
        representation of each value.
    message -- A printable object (e.g. string, number) that will be placed
        above the board in the visualisation. Default is "" (no message).
    ansi -- True if you want to use ANSI control codes to enrich the output.
        Compatible with terminals supporting ANSI control codes. Default
        False.
    compact -- True if you want to use a compact board visualisation,
        False to use a bigger one including axial coordinates along with
        the printable information in each hex. Default True (small board).
    
    Any other keyword arguments are passed through to the print function.
    Example:
        >>> board_dict = {
        ...     ( 0, 0): "hello",
        ...     ( 0, 2): "world",
        ...     ( 3,-2): "(p)",
        ...     ( 2,-1): "(S)",
        ...     (-4, 0): "(R)",
        ... }
        >>> print_board(board_dict, "message goes here", ansi=False)
        # message goes here
        #              .-'-._.-'-._.-'-._.-'-._.-'-.
        #             |     |     |     |     |     |
        #           .-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
        #          |     |     | (p) |     |     |     |
        #        .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
        #       |     |     |     | (S) |     |     |     |
        #     .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
        #    |     |     |     |     |     |     |     |     |
        #  .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
        # |     |     |     |     |hello|     |world|     |     |
        # '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
        #    |     |     |     |     |     |     |     |     |
        #    '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
        #       |     |     |     |     |     |     |     |
        #       '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
        #          |     |     |     |     |     |     |
        #          '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
        #             | (R) |     |     |     |     |
        #             '-._.-'-._.-'-._.-'-._.-'-._.-'
    """
    if compact:
        template = """# {00:}
#              .-'-._.-'-._.-'-._.-'-._.-'-.
#             |{57:}|{58:}|{59:}|{60:}|{61:}|
#           .-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
#          |{51:}|{52:}|{53:}|{54:}|{55:}|{56:}|
#        .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
#       |{44:}|{45:}|{46:}|{47:}|{48:}|{49:}|{50:}|
#     .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
#    |{36:}|{37:}|{38:}|{39:}|{40:}|{41:}|{42:}|{43:}|
#  .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
# |{27:}|{28:}|{29:}|{30:}|{31:}|{32:}|{33:}|{34:}|{35:}|
# '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
#    |{19:}|{20:}|{21:}|{22:}|{23:}|{24:}|{25:}|{26:}|
#    '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
#       |{12:}|{13:}|{14:}|{15:}|{16:}|{17:}|{18:}|
#       '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
#          |{06:}|{07:}|{08:}|{09:}|{10:}|{11:}|
#          '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
#             |{01:}|{02:}|{03:}|{04:}|{05:}|
#             '-._.-'-._.-'-._.-'-._.-'-._.-'"""
    else:
        template = """# {00:}
#                  ,-' `-._,-' `-._,-' `-._,-' `-._,-' `-.
#                 | {57:} | {58:} | {59:} | {60:} | {61:} |
#                 |  4,-4 |  4,-3 |  4,-2 |  4,-1 |  4, 0 |
#              ,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-.
#             | {51:} | {52:} | {53:} | {54:} | {55:} | {56:} |
#             |  3,-4 |  3,-3 |  3,-2 |  3,-1 |  3, 0 |  3, 1 |
#          ,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-.
#         | {44:} | {45:} | {46:} | {47:} | {48:} | {49:} | {50:} |
#         |  2,-4 |  2,-3 |  2,-2 |  2,-1 |  2, 0 |  2, 1 |  2, 2 |
#      ,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-.
#     | {36:} | {37:} | {38:} | {39:} | {40:} | {41:} | {42:} | {43:} |
#     |  1,-4 |  1,-3 |  1,-2 |  1,-1 |  1, 0 |  1, 1 |  1, 2 |  1, 3 |
#  ,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-.
# | {27:} | {28:} | {29:} | {30:} | {31:} | {32:} | {33:} | {34:} | {35:} |
# |  0,-4 |  0,-3 |  0,-2 |  0,-1 |  0, 0 |  0, 1 |  0, 2 |  0, 3 |  0, 4 |
#  `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-'
#     | {19:} | {20:} | {21:} | {22:} | {23:} | {24:} | {25:} | {26:} |
#     | -1,-3 | -1,-2 | -1,-1 | -1, 0 | -1, 1 | -1, 2 | -1, 3 | -1, 4 |
#      `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-'
#         | {12:} | {13:} | {14:} | {15:} | {16:} | {17:} | {18:} |
#         | -2,-2 | -2,-1 | -2, 0 | -2, 1 | -2, 2 | -2, 3 | -2, 4 |
#          `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-'
#             | {06:} | {07:} | {08:} | {09:} | {10:} | {11:} |
#             | -3,-1 | -3, 0 | -3, 1 | -3, 2 | -3, 3 | -3, 4 |   key:
#              `-._,-' `-._,-' `-._,-' `-._,-' `-._,-' `-._,-'     ,-' `-.
#                 | {01:} | {02:} | {03:} | {04:} | {05:} |       | input |
#                 | -4, 0 | -4, 1 | -4, 2 | -4, 3 | -4, 4 |       |  r, q |
#                  `-._,-' `-._,-' `-._,-' `-._,-' `-._,-'         `-._,-'"""
    # prepare the provided board contents as strings, formatted to size.
    ran = range(-4, +4+1)
    cells = []
    for rq in [(r,q) for r in ran for q in ran if -r-q in ran]:
        if rq in board_dict and board_dict[rq]:
            cell = ""
            for token in board_dict[rq]:
                cell = cell + token + ','
            cell = cell[:-1].center(5)
            if ansi:
                # put contents in bold
                cell = f"\033[1m{cell}\033[0m"
        else:
            cell = "     " # 5 spaces will fill a cell
        cells.append(cell)
    # prepare the message, formatted across multiple lines
    multiline_message = "\n# ".join(message.splitlines())
    # fill in the template to create the board drawing, then print!
    board = template.format(multiline_message, *cells)
    print(board, **kwargs)

In [6]:

num_tokens_upper = random.randint(0,5)

upper_positions = []
for i in range(num_tokens_upper):
    rand_x = random.randint(-4,4)
    rand_y = random.randint(-4-rand_x, 4) if rand_x <= 0 else random.randint(-4, 4-rand_x)
    symbol = list(Rules.VALID_SYMBOLS)[random.randint(0,2)]
    upper_positions.append(((rand_x,rand_y), symbol))

num_tokens_lower = random.randint(0,5)
lower_positions = []
for i in range(num_tokens_lower):
    rand_x = random.randint(-4,4)
    rand_y = random.randint(-4-rand_x, 4) if rand_x <= 0 else random.randint(-4, 4-rand_x)
    symbol = list(Rules.VALID_SYMBOLS)[random.randint(0,2)]
    lower_positions.append(((rand_x,rand_y), symbol))
    
print(upper_positions)
print(lower_positions)

[((1, -3), 'r'), ((1, 2), 'p'), ((-4, 4), 'p'), ((-1, 1), 'p'), ((2, -1), 'r')]
[((0, -1), 'p'), ((3, -2), 'r')]


In [7]:
# upper_positions = [((-1,3), 'r'), ((2,2),'r')]
# lower_positions = [((-2,3), 'p')]

upper_tokens = [Token(Hex(coordinate=position[0]), position[1]) for position in upper_positions]
lower_tokens = [Token(Hex(coordinate=position[0]), position[1]) for position in lower_positions]
team_upper = Team(UPPER)
team_lower = Team(LOWER)
team_upper.active_tokens = upper_tokens
team_upper.throws_remaining = Rules.MAX_THROWS - len(upper_tokens)
team_lower.active_tokens = lower_tokens
team_lower.throws_remaining = Rules.MAX_THROWS - len(lower_tokens)

In [8]:
state = Board(team_upper = team_upper, team_lower = team_lower)

board_dict = {}
for team_name in state.team_dict:
    team = state.team_dict[team_name]
    for token in team.active_tokens:
        _s = token.symbol
        if team_name == UPPER: 
            _s = '.'+ _s.upper() + '.'
        position = token.hex.to_tuple()
        if position in board_dict:
            board_dict[position].append(_s)
        else:
            board_dict[position] = [_s]

print(state)
# print_board(board_dict, compact=True)
print(state.evaluate(team=state.team_upper))

Team upper, throws_remaining: 4
Active tokens: ((1, -3), 'r'), ((1, 2), 'p'), ((-4, 4), 'p'), ((-1, 1), 'p'), ((2, -1), 'r'), 
Team lower, throws_remaining: 7
Active tokens: ((0, -1), 'p'), ((3, -2), 'r'), 
# 
    #              .-'-._.-'-._.-'-._.-'-._.-'-.
    #             |     |     |     |     |     |
    #           .-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
    #          |     |     |  r  |     |     |     |
    #        .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
    #       |     |     |     |  R  |     |     |     |
    #     .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
    #    |     |  R  |     |     |     |     |  P  |     |
    #  .-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-.
    # |     |     |     |  p  |     |     |     |     |     |
    # '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
    #    |     |     |     |     |  P  |     |     |     |
    #    '-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'-._.-'
    #       |     |     |     |     |     |     |     |